<a href="https://colab.research.google.com/github/green142f/3-Layer-uncrackable-cipher/blob/master/Day_9_blank_cifar10_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms, utils

In [ ]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
squeezenet = models.squeezenet1_0(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
densenet = models.densenet161(pretrained=True)
inception = models.inception_v3(pretrained=True)
googlenet = models.googlenet(pretrained=True)
shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
mobilenet = models.mobilenet_v2(pretrained=True)
resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)
mnasnet = models.mnasnet1_0(pretrained=True)

# Do not forget to enable GPUs!

In [ ]:
def use_gpu():
    torch.set_default_tensor_type(torch.cuda.FloatTensor if torch.cuda.is_available() 
                                                         else torch.FloatTensor)
use_gpu()

# You might want to use custom transforms

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

# Download the dataset

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
print(len(trainset))
print(len(testset))

50000
10000


In [ ]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print('Train Epoch: {}, Loss: {:.3f}'.format(epoch, loss.item()))


def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, pred = output.max(1)
            test_loss += criterion(output, target).item()

            correct += pred.eq(target).sum().item()
            total += target.size(0)
    test_loss /= len(test_loader.dataset)

    print('Average loss: {:.3f}, Test Acc: {:.3f} ({}/{})'.format(test_loss, 100.*correct/total, correct, total))
    
    return test_loss


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(4,4), padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(4,4), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(4,4), padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(4,4), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(4,4), padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(4,4), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        if torch.cuda.is_available():
           x = x.to(device="cuda")
        # conv layers
        x = self.conv_layer(x)

        # flatten
        x = x.view(x.size(0), -1)

        # fc layer
        x = self.fc_layer(x)

        return x

batch_size = 100
test_batch_size = 50
epochs = 20
lr = 1e-3
criterion = nn.CrossEntropyLoss()

#training
model = CNN()
optimizer = optim.Adam(model.parameters(), lr=lr)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, shuffle=False)
losses = []
for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, epoch)
    losses.append(test(model, val_loader))


Train Epoch: 1, Loss: 1.095
Average loss: 0.023, Test Acc: 58.870 (5887/10000)
Train Epoch: 2, Loss: 0.952
Average loss: 0.020, Test Acc: 65.550 (6555/10000)
Train Epoch: 3, Loss: 0.592
Average loss: 0.015, Test Acc: 74.840 (7484/10000)
Train Epoch: 4, Loss: 0.593
Average loss: 0.015, Test Acc: 74.580 (7458/10000)
Train Epoch: 5, Loss: 0.434
Average loss: 0.015, Test Acc: 76.730 (7673/10000)
Train Epoch: 6, Loss: 0.377
Average loss: 0.013, Test Acc: 79.650 (7965/10000)
Train Epoch: 7, Loss: 0.395
Average loss: 0.014, Test Acc: 78.300 (7830/10000)
Train Epoch: 8, Loss: 0.290
Average loss: 0.012, Test Acc: 81.880 (8188/10000)
Train Epoch: 9, Loss: 0.340
Average loss: 0.012, Test Acc: 82.290 (8229/10000)
Train Epoch: 10, Loss: 0.360
Average loss: 0.013, Test Acc: 81.270 (8127/10000)
Train Epoch: 11, Loss: 0.221
Average loss: 0.014, Test Acc: 81.750 (8175/10000)
Train Epoch: 12, Loss: 0.159
Average loss: 0.013, Test Acc: 82.680 (8268/10000)
Train Epoch: 13, Loss: 0.244
Average loss: 0.014,